# **IATI Datastore API data collection**

## **Dependencies**

In [64]:
import requests
import csv
import KEYS
import json
from tqdm.notebook import tqdm

## **Define relevant features**

List of all elements found here: https://developer.iatistandard.org/api-details#api=datastore&operation=query 

Description of all elements found here: https://iatistandard.org/en/iati-standard/203/activity-standard/summary-table/

In [65]:
fl_list = [
    "iati_identifier",
    "title_narrative",
    "title_narrative_xml_lang",
    "reporting_org_narrative",
    "reporting_org_ref",
    "contact_info_organisation_narrative",
    "contact_info_department_narrative",
    "description_narrative_xml_lang",
    "description_narrative",
    "recipient_country_code",
    "recipient_region_code",
    "location_name_narrative",
    "activity_status_code",
    "document_link_url",
    "last_updated_datetime",
    "activity_date_type",
    "activity_date_iso_date",
    "sector_code",
    "sector_vocabulary",
]

fl_str = ",".join(fl_list)

## **Fetch data from IATI Datastore**

In [66]:
# REF CODES

# Name | Code
# BMZ | DE-1
# Inter-American Development Bank (IAD) | XI-IATI-IADB
# Asian Development Bank (ADB) | XM-DAC-46004
# African Development Bank (AfDB) | XM-DAC-46002
# European Investment Bank (EIB) | XM-DAC-918-3
# The World Bank (WB) | 44000
# World Bank Trust Funds (WBTF) |  XI-IATI-WBTF
# ---
# Non BMZ Activity (giz-non-bmz) | XM-DAC-5-52
# Auswärtiges Amt (AA) | XM-DAC-5-7
# Kreditanstalt für Wiederaufbau (KfW) | XI-IATI-EC_INTPA-2023-PC-35363 -> not working

### Fetch as JSON

In [67]:
##########
# PARAMS #
##########

abbreviation = "aa"
code = "XM-DAC-5-7"

In [68]:
url = "https://api.iatistandard.org/datastore/activity/select"
query = {
    "q": f'reporting_org_ref:("{code}")',
    "start": 0, 
    "rows": 1000,  # max 1000 rows per page
    "fl": fl_str
}

headers = {
    "Ocp-Apim-Subscription-Key": KEYS.IATI_KEY
}

try:
    all_docs = []
    page = 0
    pbar = tqdm()

    max_pages = 1
    max_pages_avl = False
    cur_page = -1

    while True:
        if cur_page < max_pages or max_pages_avl == False:
            cur_page += 1
            print(f"Page: {page}")
            response = requests.get(url, params=query, headers=headers)
            
            if response.status_code == 200:
                data = response.json()
                docs = data.get("response", {}).get("docs", [])
                if docs:
                    all_docs.extend(docs)
                    query["start"] += len(docs)
                    page += 1
                    pbar.update(1)
                else:
                    break
            else:
                print("Error:", response.status_code)
                break
        else:
            break

    pbar.close()

    with open(f"../../src/response_{abbreviation}.json", mode='w', encoding='utf-8') as file:
        json.dump(all_docs, file, ensure_ascii=False, indent=4)
        
    print("Data downloaded and stored in src!'")

except requests.exceptions.RequestException as e:
    print("Error:", e)

0it [00:00, ?it/s]

Page: 0
Page: 1
Page: 2
Page: 3
Page: 4
Page: 5
Page: 6
Page: 7
Page: 8
Data downloaded and stored in src!'


### Fetch as CSV

In [69]:
url = "https://api.iatistandard.org/datastore/activity/select"
query = {
    "q": "*:*",
    "start": 500000,
    "rows": 1000,
    "fl": fl_str
}

headers = {
    "Ocp-Apim-Subscription-Key": KEYS.IATI_KEY2
}

try:
    response = requests.get(url, params=query, headers=headers)
    if response.status_code == 200:
        data = response.json()
        docs = data.get("response", {}).get("docs", [])
        if docs:
            with open("../data/response_iati.csv", mode='w', newline='', encoding='utf-8') as file:
                writer = csv.DictWriter(file, fieldnames=fl_list)
                writer.writeheader()
                for doc in docs:
                    row = {key: doc.get(key, '') for key in fl_list}
                    writer.writerow(row)
            print("Data downlaoded and stored as 'response_iati.csv'")
        else:
            print("No data in response!")
    else:
        print("Error:", response.status_code)
except requests.exceptions.RequestException as e:
    print("Error:", e)

FileNotFoundError: [Errno 2] No such file or directory: '../data/response_iati.csv'